# 数据加载

In [6]:
import os
import codecs
import time
import sys
import numpy as np
import tensorflow as tf
import data_util
import similarity
from tensorflow import keras
from tensorflow.keras import layers

# data loading params
tf.compat.v1.flags.DEFINE_string(
    "knowledge_file", "data/knowledge.txt", "Knowledge data.")
tf.compat.v1.flags.DEFINE_string(
    "train_file", "data/train.txt", "Training data.")
tf.compat.v1.flags.DEFINE_string(
    "test_file", "data/test.txt", "Test data.")
tf.compat.v1.flags.DEFINE_string(
    "stop_words_file", "data/stop_words.txt", "Stop words.")

# result & model save params
tf.compat.v1.flags.DEFINE_string(
    "result_file", "res/predictRst.score", "Predict result.")
tf.compat.v1.flags.DEFINE_string(
    "save_file", "res/savedModel", "Save model.")

# pre-trained word embedding vectors
# Path to embedding file!
tf.compat.v1.flags.DEFINE_string("embedding_file", 
  "zhwiki_2017_03.sg_50d_1.word2vec", 
  "Embedding vectors.")

# hyperparameters
tf.compat.v1.flags.DEFINE_integer(
    "k", 5, "K most similarity knowledge (default: 5).")
tf.compat.v1.flags.DEFINE_integer(
    "rnn_size", 100, 
    "Neurons number of hidden layer in LSTM cell (default: 100).")
tf.compat.v1.flags.DEFINE_float(
    "margin", 0.1, "Constant of max-margin loss (default: 0.1).")
tf.compat.v1.flags.DEFINE_integer(
    "max_grad_norm", 5, "Control gradient expansion (default: 5).")
tf.compat.v1.flags.DEFINE_integer(
    "embedding_dim", 50, "Dimensionality of character embedding (default: 50).")
tf.compat.v1.flags.DEFINE_integer(
    "max_sentence_len", 100, "Maximum number of words in a sentence (default: 100).")
tf.compat.v1.flags.DEFINE_float(
    "dropout_keep_prob", 0.45, "Dropout keep probability (default: 0.5).")
tf.compat.v1.flags.DEFINE_float(
    "learning_rate", 0.001, "Learning rate (default: 0.4).")
tf.compat.v1.flags.DEFINE_float(
    "lr_down_rate", 0.5, "Learning rate down rate(default: 0.5).")
tf.compat.v1.flags.DEFINE_integer(
    "lr_down_times", 4, "Learning rate down times (default: 4)")
tf.compat.v1.flags.DEFINE_float(
    "l2_reg_lambda", 0.1, "L2 regularization lambda (default: 0.0)")

# training parameters
tf.compat.v1.flags.DEFINE_integer(
    "batch_size", 512, "Batch Size (default: 64)")
tf.compat.v1.flags.DEFINE_integer(
    "num_epochs", 20, "Number of training epochs (default: 20)")
tf.compat.v1.flags.DEFINE_integer(
    "evaluate_every", 20, "Evaluate model on dev set after this many steps (default: 100)")
tf.compat.v1.flags.DEFINE_integer(
    "checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.compat.v1.flags.DEFINE_integer(
    "num_checkpoints", 20, "Number of checkpoints to store (default: 5)")

# gpu parameters
tf.compat.v1.flags.DEFINE_float(
    "gpu_mem_usage", 0.75, "GPU memory max usage rate (default: 0.75).")
tf.compat.v1.flags.DEFINE_string(
    "gpu_device", "/gpu:0", "GPU device name.")

# misc parameters
tf.compat.v1.flags.DEFINE_boolean(
    "allow_soft_placement", True, "Allow device soft device placement.")

#加了一行代码,以适应jupyter
tf.compat.v1.flags.DEFINE_string('f', '', 'kernel')

FLAGS = tf.compat.v1.flags.FLAGS
FLAGS(sys.argv)

['/usr/local/miniconda3/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py']

In [7]:
embedding, word2idx = data_util.load_embedding(FLAGS.embedding_file)
stop_words = codecs.open(FLAGS.stop_words_file, 'r', encoding='utf8').readlines()
stop_words = [w.strip() for w in stop_words]
#similarity.generate_dic_and_corpus(FLAGS.knowledge_file, FLAGS.train_file, 
#                                    stop_words)
train_sim_ixs = similarity.topk_sim_ix(FLAGS.train_file, stop_words, FLAGS.k)
test_sim_ixs = similarity.topk_sim_ix(FLAGS.test_file, stop_words, FLAGS.k)

In [8]:
train_questions, train_answers, train_labels, train_question_num =        \
    data_util.load_data(FLAGS.knowledge_file, FLAGS.train_file, word2idx, \
    stop_words, train_sim_ixs, FLAGS.max_sentence_len)

test_questions, test_answers, test_labels, test_question_num =            \
    data_util.load_data(FLAGS.knowledge_file, FLAGS.test_file, word2idx,  \
    stop_words, test_sim_ixs, FLAGS.max_sentence_len)

questions, true_answers, false_answers = [], [], []

for q, ta, fa in data_util.training_batch_iter(
    train_questions, train_answers, train_labels, 
    train_question_num, FLAGS.batch_size
):
    #questions.append(q), true_answers.append(ta), false_answers.append(fa)
    #用于分布式训练
    questions.extend(q), true_answers.extend(ta), false_answers.extend(fa)

t_questions, t_answers = [], []  
for q, a in data_util.testing_batch_iter(
    test_questions, test_answers, test_question_num, FLAGS.batch_size
):
    #questions.append(q), true_answers.append(ta), false_answers.append(fa)
    #用于分布式训练
    t_questions.extend(q), t_answers.extend(a)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.987 seconds.
Prefix dict has been built successfully.


In [9]:
strategy = tf.distribute.MirroredStrategy()
print ('Number of devices: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of devices: 1


In [48]:
GLOBAL_BATCH_SIZE = FLAGS.batch_size * strategy.num_replicas_in_sync
GLOBAL_BATCH_SIZE

512

In [49]:
train_dataset = tf.data.Dataset.from_tensor_slices((questions, true_answers, false_answers)).batch(GLOBAL_BATCH_SIZE) 
test_dataset = tf.data.Dataset.from_tensor_slices((t_questions, t_answers)).batch(GLOBAL_BATCH_SIZE) 

train_dist_dataset = strategy.experimental_distribute_dataset(train_dataset)
test_dist_dataset = strategy.experimental_distribute_dataset(test_dataset)


In [50]:
def simModel(embedding, embedding_dim, vocab_size, max_sentence_len, 
    rnn_size, dropout_keep_prob):
    
    class SimLayer(layers.Layer):
        def __init__(self):
            super(SimLayer, self).__init__()

        def call(self, q, a):
            q1 = tf.sqrt(tf.reduce_sum(tf.multiply(q, q), 1))
            a1 = tf.sqrt(tf.reduce_sum(tf.multiply(a, a), 1))
            mul = tf.reduce_sum(tf.multiply(q, a), 1)
            cosSim = tf.divide(mul, tf.multiply(q1, a1))
            return cosSim
    
    bilstm_inputs = layers.Input(shape=(max_sentence_len,))

    emb = layers.Embedding(vocab_size, embedding_dim, 
       weights=[np.asarray(embedding)], trainable=False)  ##初始化词向量
    emb_inputs = emb(bilstm_inputs)
    
    bilstm = layers.Bidirectional(
      layers.LSTM(
          rnn_size, activation='relu', return_sequences=True,
          dropout=dropout_keep_prob, recurrent_dropout=dropout_keep_prob
      ))(emb_inputs)

    bilstm_max = tf.keras.backend.max(bilstm, axis=1)
    bilstm_out = tf.keras.backend.tanh(bilstm_max)
    bilstm_model = keras.Model(inputs=bilstm_inputs, outputs=bilstm_out)
    bilstm_model.summary()
    
    q_inputs = layers.Input(shape=(max_sentence_len,))
    a_inputs = layers.Input(shape=(max_sentence_len,))
    q_bilstm = bilstm_model(q_inputs)
    a_bilstm = bilstm_model(a_inputs)
    
    similarity = SimLayer()(q_bilstm, a_bilstm)
    sim_model = keras.Model(inputs=[q_inputs, a_inputs], outputs=similarity)
    
    sim_model.summary()
    return sim_model

In [51]:
# 创建检查点目录以存储检查点。
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

In [52]:
with strategy.scope():

  def compute_loss(trueCosSim, falseCosSim, margin): #在一个GLobal_batch下计算损失
    zero = tf.fill(tf.shape(trueCosSim), 0.0)
    tfMargin = tf.fill(tf.shape(trueCosSim), margin)
   
    # max-margin losses = max(0, margin - true + false)
    per_example_loss = tf.maximum(zero, tf.subtract(tfMargin, 
      tf.subtract(trueCosSim, falseCosSim))) #[global_batch]
    #loss = tf.reduce_sum(losses)
    return tf.nn.compute_average_loss(per_example_loss, 
                                      global_batch_size=GLOBAL_BATCH_SIZE)

### 训练循环

必须在'strategy.scope'下创建模型和优化器

In [57]:
#采用sim_model作为最终模型
with strategy.scope():
    vocab_size = len(embedding)
    sim_model = simModel(embedding, FLAGS.embedding_dim, vocab_size, 
        FLAGS.max_sentence_len, FLAGS.rnn_size, FLAGS.dropout_keep_prob)
    margin = FLAGS.margin

    optimizer = tf.keras.optimizers.Adam(lr = FLAGS.learning_rate)

    checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=sim_model)

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        [(None, 100)]             0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 100, 50)           29922700  
_________________________________________________________________
bidirectional_6 (Bidirection (None, 100, 200)          120800    
_________________________________________________________________
tf_op_layer_Max_6 (TensorFlo [(None, 200)]             0         
_________________________________________________________________
tf_op_layer_Tanh_6 (TensorFl [(None, 200)]             0         
Total params: 30,043,500
Trainable params: 120,800
Non-trainable params: 29,922,700
_________________________________________________________________
Model: "model_13"
________________________________________________________________________________________

In [58]:
with strategy.scope():
    #一个global_batch数据分发到各个机器
    def train_step(inputs):
        question, trueAnswer, falseAnswer = inputs
        #print('train_step',question.shape, trueAnswer.shape, falseAnswer.shape)
        question = tf.cast(question, dtype='float32')
        trueAnswer = tf.cast(trueAnswer, dtype='float32')
        falseAnswer = tf.cast(falseAnswer, dtype='float32')
        
        with tf.GradientTape() as tape:

            trueCosSim = sim_model((question, trueAnswer))
            falseCosSim = sim_model((question, falseAnswer))
        
            loss = compute_loss(trueCosSim, falseCosSim, margin)
        grads = tape.gradient(loss, sim_model.trainable_variables)
        optimizer.apply_gradients(zip(grads, sim_model.trainable_variables))
        #loss_metric(loss)
        #tf.print('mean loss = %s' % (loss))
        return loss
    
    def test_step(inputs):
        test_q, test_a = inputs
        test_q = tf.cast(test_q, dtype='float32')
        test_a = tf.cast(test_a, dtype='float32')
        cosSim = sim_model((test_q, test_a))
        return cosSim

In [60]:
with strategy.scope():
    # `experimental_run_v2`将复制提供的计算并使用分布式输入运行它。
    @tf.function
    #一个global_batch数据分发到各个机器
    def distributed_train_step(dataset_inputs):
        #分布式训练，返回聚合损失
        per_replica_losses = strategy.experimental_run_v2(train_step,
          args=(dataset_inputs,))
        return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses,
                   axis=None)
    
    #@tf.function
    def distributed_test_step(test_dataset):
        t_labels = tf.constant(test_labels)
        first_time = True
        scores = tf.constant(np.array([], dtype='float32'))
        for x in test_dataset:
            #print(x)
            score = strategy.experimental_run_v2(test_step, args=(x,))
            scores = tf.concat((scores, score), axis=0)      
        cnt = 0
        scores = tf.abs(scores)
        for test_id in tf.range(test_question_num):    
            offset = tf.multiply(test_id, 4)          
            predict_true_ix = tf.argmax(scores[offset:tf.add(offset,4)], output_type=offset.dtype)
            if t_labels[tf.add(offset,predict_true_ix)] == 1:
                cnt += 1
        tf.print("evaluation acc: ", tf.divide(cnt, test_question_num))

    for epoch in range(FLAGS.num_epochs):
        # 训练循环
        total_loss = 0.0
        num_batches = 0
        for x in train_dist_dataset:
            batch_loss = distributed_train_step(x)
            total_loss += batch_loss
            #tf.print('global_batch_loss: ', batch_loss * 1000)
            num_batches += 1
            train_loss = total_loss / num_batches              
        template = ("Epoch {}, Loss: {}")
        tf.print (template.format(epoch+1, train_loss))
        # 测试循环
        distributed_test_step(test_dist_dataset)
        if epoch % 2 == 0:
            checkpoint.save(checkpoint_prefix)

Epoch 1, Loss: 0.09830742329359055
evaluation acc:  0.2859244727807749
Epoch 2, Loss: 0.09824561327695847
evaluation acc:  0.28347229033840116
Epoch 3, Loss: 0.09816975146532059
evaluation acc:  0.28249141736145167
Epoch 4, Loss: 0.09812230616807938
evaluation acc:  0.28347229033840116
Epoch 5, Loss: 0.09811707586050034
evaluation acc:  0.2805296714075527
Epoch 6, Loss: 0.09795045852661133
evaluation acc:  0.2790583619421285
Epoch 7, Loss: 0.0978194922208786
evaluation acc:  0.2829818538499264
Epoch 8, Loss: 0.09785612672567368
evaluation acc:  0.2829818538499264
Epoch 9, Loss: 0.09773333370685577
evaluation acc:  0.2864149092692496
Epoch 10, Loss: 0.09837506711483002
evaluation acc:  0.28249141736145167
Epoch 11, Loss: 0.09931309521198273
evaluation acc:  0.2653261402648357
Epoch 12, Loss: 0.09923665970563889
evaluation acc:  0.2741539970573811
Epoch 13, Loss: 0.09909264743328094
evaluation acc:  0.28249141736145167
Epoch 14, Loss: 0.09893905371427536
evaluation acc:  0.28200098087297